In [63]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [58]:
temp = pd.read_csv('Bitstamp_BTCUSD_2017_minute.csv', header = 1)
for i in range(2018,2022):
    df = pd.read_csv(f'Bitstamp_BTCUSD_{i}_minute.csv',header=1)
    print(df.head())
    j =+ len(df)
    print(j)
    
    temp = pd.concat([temp,df],axis = 0, ignore_index = True)
df = temp

         unix                 date   symbol     open     high      low  \
0  1546300740  2018-12-31 23:59:00  BTC/USD  3689.26  3693.30  3689.26   
1  1546300680  2018-12-31 23:58:00  BTC/USD  3689.26  3691.35  3689.26   
2  1546300620  2018-12-31 23:57:00  BTC/USD  3688.83  3689.26  3688.83   
3  1546300560  2018-12-31 23:56:00  BTC/USD  3687.87  3689.65  3686.92   
4  1546300500  2018-12-31 23:55:00  BTC/USD  3688.28  3688.85  3685.00   

     close  Volume BTC    Volume USD  
0  3693.30    9.838855  36337.841473  
1  3691.35    0.560000   2067.156000  
2  3689.26    0.560833   2069.059602  
3  3686.92    7.610240  28058.346208  
4  3688.85    7.665703  28277.629729  
525599
         unix                 date   symbol     open     high      low  \
0  1577836740  2019-12-31 23:59:00  BTC/USD  7167.30  7171.22  7167.30   
1  1577836680  2019-12-31 23:58:00  BTC/USD  7166.89  7167.30  7161.99   
2  1577836620  2019-12-31 23:57:00  BTC/USD  7164.22  7170.80  7161.65   
3  1577836560  201

In [59]:
df = df.drop(['unix','symbol','Volume BTC'], axis = 1,errors='ignore')

In [60]:
from sklearn.preprocessing import MinMaxScaler

data_training = df[df['date'] < '2020-01-01 00:01:00'].drop('date', axis = 1,errors='ignore')
data_testing = df[df['date'] > '2020-01-01 00:01:00'].drop('date', axis = 1,errors='ignore')

#data_training = df.drop('date', axis = 1,errors='ignore')
#data_testing = df.drop('date', axis = 1,errors='ignore')

In [61]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)
data_training

array([[6.95857408e-01, 6.95835552e-01, 6.94010462e-01, 6.94097562e-01,
        1.06397168e-03],
       [6.95856350e-01, 6.97979558e-01, 6.94936985e-01, 6.97997907e-01,
        2.52840307e-03],
       [6.95616842e-01, 6.95834494e-01, 6.94423720e-01, 6.95856075e-01,
        1.39682328e-03],
       ...,
       [1.55487328e-01, 1.55449187e-01, 1.55407114e-01, 1.55459188e-01,
        1.41895586e-04],
       [1.55322368e-01, 1.55284222e-01, 1.55317161e-01, 1.55156233e-01,
        4.61311541e-04],
       [1.55586198e-01, 1.55548059e-01, 1.55530404e-01, 1.55369306e-01,
        4.54189902e-03]])

In [66]:
X_train = [] 
Y_train = []
data_training.shape[0]
for i in range(120, data_training.shape[0]):
    X_train.append(data_training[i-120:i])
    Y_train.append(data_training[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train.shape

(1576677, 120, 5)

In [71]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
#Initialize the RNN
model = Sequential() 
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
model.add(Dropout(0.2)) 
model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3)) 
model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4)) 
model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5)) 
model.add(Dense(units =1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 120, 50)           11200     
_________________________________________________________________
dropout (Dropout)            (None, 120, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 120, 60)           26640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 60)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 120, 80)           45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 120, 80)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               9

In [72]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
history= model.fit(X_train, Y_train, epochs = 10, batch_size =50, validation_split=0.1)

Epoch 1/10
 1744/28381 [>.............................] - ETA: 1:17:47 - loss: 0.0021

KeyboardInterrupt: 

In [ ]:
ii